Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Using a Trained Model for Batch Inference

In this notebook, we will demonstrate how to make predictions on large quantities of data asynchronously using the ML pipelines with Azure Machine Learning. Batch inference (or batch scoring) provides cost-effective inference, with unparalleled throughput for asynchronous applications. Batch prediction pipelines can scale to perform inference on terabytes of production data. Batch prediction is optimized for high throughput, fire-and-forget predictions for a large collection of data.

> **Tip**
The dataset we use is not that huge. We aim to make you know the workflow of batch inference. If your system requires low-latency processing (to process a single document or small set of documents quickly), please use realtime inference. Refer to fasttext_realtime_inference.ipynb for more details. 

The outline of this notebook is as follows:

- Create a DataStore referencing documents stored in a blob container.
- Reference a trained fastText model from a complete experiment.
- Use the fastText model to do batch inference on the documents in the data blob container.

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at https://github.com/Azure/MachineLearningNotebooks first. This sets you up with a working config file that has information on your workspace, subscription id, etc.

In [1]:
from azureml.core import Workspace, Dataset, Datastore, Run
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.data.datapath import DataPath
from azureml.core.experiment import Experiment
from azureml.pipeline.wrapper import PipelineRun, Module, dsl

### Connect to workspace
Create a workspace object from the existing workspace. Workspace.from_config() reads the file config.json and loads the details into an object named workspace.


In [2]:
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

DesignerDRI_EASTUS
DesignerDRI
eastus
74eccef0-4b8d-4f83-b5f9-fa100d155b22
dict_keys(['attached-aks', 'default', 'compute', 'aml-compute', 'aml-compute-gpu'])


### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes. If the AmlCompute with that name is already in your workspace the code will skip the creation process.**


In [3]:
aml_compute_name = 'aml-compute'
if aml_compute_name in workspace.compute_targets:
    aml_compute = AmlCompute(workspace, aml_compute_name)
    print("Found existing compute target: {}".format(aml_compute_name))
else:
    print("Creating new compute target: {}".format(aml_compute_name))
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=1, max_nodes=4)
    aml_compute = ComputeTarget.create(workspace, aml_compute_name, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target: aml-compute


### Upload the dataset onto a blob container and register it to the workspace.

In [4]:
dataset_name = 'THUCNews_For_Batch_Inference'
# if the workspace don't contain the dataset, then register it
if not dataset_name in workspace.datasets:
    # upload files onto path_on_datastore to a blob container
    # our files are in the directory of 'path_on_datastore' in the blob container
    path_on_datastore = 'data_for_batch_inference'
    datastore = Datastore.get(workspace=workspace, datastore_name='workspaceblobstore')
    datastore.upload(src_dir='data/data_for_batch_inference', target_path=path_on_datastore, overwrite=True, show_progress=True)
    # description of the dataset
    description = 'THUCNews dataset is generated by filtering and filtering historical data \
    of Sina News RSS subscription channel from 2005 to 2011'
    # get the DataPath object associated with the uploaded dataset
    datastore_path = [DataPath(datastore=datastore, path_on_datastore=path_on_datastore)]
    data = Dataset.File.from_files(path=datastore_path)
    # register the dataset to your workspace
    data.register(workspace=workspace, name=dataset_name, description=description, create_new_version=True)
# get the registered dataset
dataset = workspace.datasets[dataset_name]

### Register an anonymous module from yaml file to the workspace.
If you decorate your module function with ```@dsl.module```, azure-cli-ml could help to generate the ```*.spec.yaml``` file.

In [5]:
fasttext_score_module_func = Module.from_yaml(workspace, 'fasttext_score/fasttext_score.spec.yaml')

### Load a trained fastText model from a complete experiment
- get all experiments
- choose an experiment from all experiments
- get the latest run
- get a PipelineRun associated with the run

In [6]:
exp_name_list = [exp.name for exp in Experiment.list(workspace)]
exp_name_list

['sample10',
 'sample5',
 'sample5-realtime',
 'simple10-batch',
 'pythonscript',
 'Data_dependency',
 'clement',
 'new_module',
 'test_module2',
 'test_m',
 'module_SDK_local_module_test',
 'fasttext_pipeline',
 'fasttext_batch_inference',
 'fasttext_pipeline2']

### Choose the experiment you want with its name

In [7]:
experiment_name = "fasttext_pipeline"
experiment = Experiment(workspace, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
fasttext_pipeline,DesignerDRI_EASTUS,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
# azureml.pipeline.core.run.PipelineRun
run = Run.list(experiment, status='Completed').__next__()
run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,0a6a356c-39e9-41b5-a486-af5a0ea72c05,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Get a PipelineRun object

In [9]:
run_id = run.id
# azureml.pipeline.wrapper._pipeline_run.PipelineRun
pipeline_run = PipelineRun(experiment, run_id)
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,0a6a356c-39e9-41b5-a486-af5a0ea72c05,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Visualize the pipeline so as to obtain information about the module

In [10]:
pipeline_run.visualize()

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_32946f55-045b-48eb-8046-54013b6623e1_widget', env_json='{}', graph_jso…

### Get the trained model from a StepRun object.
- get a StepRun from the PipelineRun
- get the port with the trained model from the StepRun
- get DataPath from the port
- change DataPath into the form of module input

In [11]:
# obtain step_run_id from the visualization result.
step_run_id = '845ab0f8-eac2-49b4-b3fc-6cc6f1541ee7'
step_run = pipeline_run.get_step_run(step_run_id)
step_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,845ab0f8-eac2-49b4-b3fc-6cc6f1541ee7,azureml.StepRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
print(type(step_run))
print(type(data_path))

<class 'azureml.pipeline.wrapper._pipeline_run.StepRun'>
<class 'azureml.data.datapath.DataPath'>


In [14]:
data_path = port.get_data_path()
data_path

### Use the trained model as the input of a new pipeline

In [15]:
# name will be from the visualization result.
# get_port() should supports three kinds of names: (1)the_better_model (2)The better model (3)The_better_model
port = step_run.get_port(name='The_better_model')
data_path = port.get_data_path()
model = Dataset.File.from_files(path=[data_path]).as_named_input('model_for_batch_inference')
model

### Construct the pipeline

In [16]:
@dsl.pipeline(name='batch inference', description='Batch Inference', default_compute_target=aml_compute.name)
def training_pipeline():
    fasttext_score = fasttext_score_module_func(
        texts_to_score=dataset,
        fasttext_model_dir=model
    )
    fasttext_score.runsettings.configure(node_count=1, process_count_per_node=2, mini_batch_size="64")

    return {**fasttext_score.outputs}

In [17]:
# pipeline
pipeline = training_pipeline()
# pipeline.save(experiment_name=experiment_name)
pipeline

In [19]:
# pipeline_run
experiment_name = 'fasttext_batch_inference'
pipeline_run = pipeline.submit(experiment_name=experiment_name, regenerate_outputs=True)
pipeline_run.wait_for_completion()

Submitted PipelineRun cef086df-436b-46c1-991a-0bb5d85039b5
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_batch_inference/runs/cef086df-436b-46c1-991a-0bb5d85039b5?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/DesignerDRI/workspaces/DesignerDRI_EASTUS
PipelineRunId: cef086df-436b-46c1-991a-0bb5d85039b5
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_batch_inference/runs/cef086df-436b-46c1-991a-0bb5d85039b5?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/DesignerDRI/workspaces/DesignerDRI_EASTUS


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_65801a16-883e-481c-bc1c-87059c6fb25b_widget', env_json='{}', graph_jso…

<RunStatus.completed: 'Completed'>

### Download results of batch